In [2]:
import heapq
from collections import Counter
import cv2
import numpy as np

# Node class for Huffman tree
class Node:
    def __init__(self, freq, pixel, left=None, right=None):
        self.freq = freq
        self.pixel = pixel
        self.left = left
        self.right = right

    # Define comparison for priority queue
    def __lt__(self, other):
        return self.freq < other.freq

# Build Huffman Tree
def build_huffman_tree(frequency):
    heap = [Node(freq, pixel) for pixel, freq in frequency.items()]
    heapq.heapify(heap)
    
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        merged = Node(left.freq + right.freq, None, left, right)
        heapq.heappush(heap, merged)
    
    return heap[0]

# Generate Huffman Codes
def generate_huffman_codes(node, current_code="", codes={}):
    if node is None:
        return
    
    if node.pixel is not None:
        codes[node.pixel] = current_code
    
    generate_huffman_codes(node.left, current_code + "0", codes)
    generate_huffman_codes(node.right, current_code + "1", codes)
    
    return codes

# Encode Image
def huffman_encoding(image, codes):
    encoded_image = "".join(codes[pixel] for row in image for pixel in row)
    return encoded_image

# Decode Image
def huffman_decoding(encoded_image, root, shape):
    decoded_image = []
    current_node = root
    for bit in encoded_image:
        if bit == "0":
            current_node = current_node.left
        else:
            current_node = current_node.right
            
        if current_node.pixel is not None:
            decoded_image.append(current_node.pixel)
            current_node = root
    
    return np.array(decoded_image).reshape(shape)

# Main function to apply Huffman coding to an image
def main():
    # Load the image in grayscale mode
    image = cv2.imread('lenna.jpeg', cv2.IMREAD_GRAYSCALE)
    
    # Step 1: Calculate Frequency
    frequency = Counter(image.flatten())
    
    # Step 2: Build Huffman Tree
    root = build_huffman_tree(frequency)
    
    # Step 3: Generate Huffman Codes
    codes = generate_huffman_codes(root)
    
    # Step 4: Encode Image
    encoded_image = huffman_encoding(image, codes)
    
    # Step 5: Decode Image
    decoded_image = huffman_decoding(encoded_image, root, image.shape)
    
    # Display the original and decoded images
    cv2.imshow('Original Image', image)
    cv2.imshow('Decoded Image', decoded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
